In [19]:
import requests 
import pandas as pd
from datetime import datetime,date 

In [20]:
def fetch_crypto_data(crypto_pair: str, start_date: str) -> pd.DataFrame:
    """
    Fetches historical cryptocurrency data from the CoinGecko API.
    
    Parameters:
    - crypto_pair (str): The cryptocurrency pair to fetch data for (e.g., "bitcoin/usd").
    - start_date (str): The start date for fetching historical data in 'YYYY-MM-DD' format.
    
    Returns:
    - pd.DataFrame: A DataFrame containing historical price data with columns: Date, Open, High, Low, Close.
    """
    
    # Split the crypto_pair into its components: crypto_id and currency
    crypto_id, currency = crypto_pair.split('/')
    
    # Convert the start date to a UNIX timestamp
    start_timestamp = int(datetime.strptime(start_date, '%Y-%m-%d').timestamp())
    
    # Construct the API URL for fetching market chart data within a specific date range
    url = f"https://api.coingecko.com/api/v3/coins/{crypto_id}/market_chart/range"
    
    # Set up parameters for the API request
    params = {
        'vs_currency': currency.lower(),  # The currency to compare against (e.g., 'usd')
        'from': start_timestamp,           # Start timestamp
        'to': int(datetime.now().timestamp()),  # Current timestamp
    }
    
    # Send a GET request to the CoinGecko API
    response = requests.get(url, params=params)
    
    # Check if the response status is not OK (200)
    if response.status_code != 200:
        raise Exception(f"Error fetching data: {response.status_code} - {response.text}")

    # Parse the JSON response
    data = response.json()

    # Check if 'prices' are present in the response data
    if 'prices' not in data:
        raise Exception("No price data found in the response.")

    # Create a DataFrame from the price data (timestamps and closing prices)
    df = pd.DataFrame(data['prices'], columns=['timestamp', 'close'])
    
    # Convert the timestamp from milliseconds to a date format and add it to the DataFrame
    df['Date'] = pd.to_datetime(df['timestamp'], unit='ms').dt.date
    
    # Initialize Open, High, Low, and Close columns with closing prices
    df['Open'] = df['close']
    df['High'] = df['close']
    df['Low'] = df['close']
    df['Close'] = df['close']

    # Group the DataFrame by date and aggregate the Open, High, Low, and Close values
    df = df.groupby('Date').agg({
        'Open': 'first',  # First entry for Open price
        'High': 'max',    # Maximum price for High
        'Low': 'min',     # Minimum price for Low
        'Close': 'last'   # Last entry for Close price
    }).reset_index()

    return df

# Example usage
if __name__ == "__main__":
    crypto_pair = "bitcoin/usd"
    
    from datetime import timedelta
    start_date = (datetime.today() - timedelta(days=360)).strftime('%Y-%m-%d')

    try:
        crypto_data = fetch_crypto_data(crypto_pair, start_date)
        print(crypto_data)
    except Exception as e:
        print(e)


           Date           Open           High            Low          Close
0    2024-07-07   58230.484154   58230.484154   58230.484154   58230.484154
1    2024-07-08   55880.375378   55880.375378   55880.375378   55880.375378
2    2024-07-09   56665.140734   56665.140734   56665.140734   56665.140734
3    2024-07-10   57988.303512   57988.303512   57988.303512   57988.303512
4    2024-07-11   57704.202284   57704.202284   57704.202284   57704.202284
..          ...            ...            ...            ...            ...
356  2025-06-28  107078.915606  107078.915606  107078.915606  107078.915606
357  2025-06-29  107331.585485  107331.585485  107331.585485  107331.585485
358  2025-06-30  108396.616313  108396.616313  108396.616313  108396.616313
359  2025-07-01  107132.799107  107132.799107  107132.799107  107132.799107
360  2025-07-02  105613.399742  105613.399742  105613.399742  105613.399742

[361 rows x 5 columns]


In [21]:
today = date.today()
print("Today's date:", today)
if(today in crypto_data['Date'].values):
    today_data = crypto_data[crypto_data['Date'] == today]
    print("Today's data:", today_data)
else:
    print("No data available for today:", today)

Today's date: 2025-07-02
Today's data:            Date           Open           High            Low          Close
360  2025-07-02  105613.399742  105613.399742  105613.399742  105613.399742


In [22]:
def calculate_crypto_metrics(df,variable1,variable2):
    #Rolling window calculations for high and low 
    df['High_Last_{}_Days'.format(variable1)] = df['High'].rolling(window=variable1,min_periods=1).max()
    df['Low_Last_{}_Days'.format(variable1)] = df['Low'].rolling(window=variable1,min_periods=1).min()
    
    #Function to get the index of the last maximum or minimum 
    
    def last_max_index(series):
        return series[::-1].idxmax()
    
    def last_min_index(series):
        return series[::-1].idxmin()
    
    #calculate days since last high 
    last_high_indices = df['High'].rolling(window=variable1, min_periods=1).apply(last_max_index, raw=False)
    df[f'Days_Since_Last_High_{variable1}_Days'] = (df.index - last_high_indices)
    
    #calculate days since last low
    last_low_indices = df['Low'].rolling(window=variable1, min_periods=1).apply(last_min_index, raw=False)
    df[f'Days_Since_Last_Low_{variable1}_Days'] = (df.index - last_low_indices)
    
    #Calculate the percentage change between the high and low
    df['%_Diff_From_High_Last_{}_Days'.format(variable1)] = ((df['Close'] - df['Low_Last_{}_Days'.format(variable1)]) / df['Low_Last_{}_Days'.format(variable1)]) * 100
    
    df['%_Diff_From_Low_Last_{}_Days'.format(variable1)] = ((df['Close'] - df['High_Last_{}_Days'.format(variable1)]) / df['High_Last_{}_Days'.format(variable1)]) * 100
    
    #Future High and Low calculations
    df['High_Next_{}_Days'.format(variable2)] = df['High'].shift(-variable2).rolling(window=variable2, min_periods=1).max()
    df['Low_Next_{}_Days'.format(variable2)] = df['Low'].shift(-variable2).rolling(window=variable2, min_periods=1).min()
    
    #If Nan,then use the previous high value
    df['High_Next_{}_Days'.format(variable2)].fillna(method='ffill', inplace=True)
    df['Low_Next_{}_Days'.format(variable2)].fillna(method='ffill', inplace=True)
    
    # Percentage Difference from Future High and Low
    df['%_Diff_From_High_Next_{}_Days'.format(variable2)] = (
        (df['Close'] - df['High_Next_{}_Days'.format(variable2)]) / df['High_Next_{}_Days'.format(variable2)] * 100
    )

    df['%_Diff_From_Low_Next_{}_Days'.format(variable2)] = (
        (df['Close'] - df['Low_Next_{}_Days'.format(variable2)]) / df['Low_Next_{}_Days'.format(variable2)] * 100
    )
    
    return df
    

In [24]:
#set your variables1 and variable2
variable1 = 7
variable2=5

#call the function to calculate metrics
df=calculate_crypto_metrics(crypto_data,variable1,variable2)

C:\Users\Anshu\AppData\Local\Temp\ipykernel_16960\3865999827.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['High_Next_{}_Days'.format(variable2)].fillna(method='ffill', inplace=True)
C:\Users\Anshu\AppData\Local\Temp\ipykernel_16960\3865999827.py:32: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df['High_Next_{}_Days'.format(variable2)].fillna(method='ffill', inplace=True)
C:\Users\Anshu\AppData\Local\Temp\ipykernel_16960\3865999

In [25]:
df

,Date,Open,High,Low,Close,High_Last_7_Days,Low_Last_7_Days,Days_Since_Last_High_7_Days,Days_Since_Last_Low_7_Days,%_Diff_From_High_Last_7_Days,%_Diff_From_Low_Last_7_Days,High_Next_5_Days,Low_Next_5_Days,%_Diff_From_High_Next_5_Days,%_Diff_From_Low_Next_5_Days
0,2024-07-07,58230.484154,58230.484154,58230.484154,58230.484154,58230.484154,58230.484154,0.0,0.0,0.000000,0.000000,57388.192479,57388.192479,1.467709,1.467709
1,2024-07-08,55880.375378,55880.375378,55880.375378,55880.375378,58230.484154,55880.375378,1.0,0.0,0.000000,-4.035874,57899.288957,57388.192479,-3.486940,-2.627400
2,2024-07-09,56665.140734,56665.140734,56665.140734,56665.140734,58230.484154,55880.375378,2.0,1.0,1.404367,-2.688185,59152.719139,57388.192479,-4.205349,-1.259931
3,2024-07-10,57988.303512,57988.303512,57988.303512,57988.303512,58230.484154,55880.375378,3.0,2.0,3.772215,-0.415900,60942.327740,57388.192479,-4.847245,1.045705
4,2024-07-11,57704.202284,57704.202284,57704.202284,57704.202284,58230.484154,55880.375378,4.0,3.0,3.263806,-0.903791,64835.489733,57388.192479,-10.999049,0.550653
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,2025-06-28,107078.915606,107078.915606,107078.915606,107078.915606,107238.530450,100852.582646,2.0,5.0,6.173697,-0.148841,108396.616313,105613.399742,-1.215629,1.387623
357,2025-06-29,107331.585485,107331.585485,107331.585485,107331.585485,107331.585485,100852.582646,0.0,6.0,6.424231,0.000000,108396.616313,105613.399742,-0.982531,1.626863
358,2025-06-30,108396.616313,108396.616313,108396.616313,108396.616313,108396.616313,105511.624379,0.0,6.0,2.734288,0.000000,107132.799107,105613.399742,1.179673,2.635287
359,2025-07-01,107132.799107,107132.799107,107132.799107,107132.799107,108396.616313,105976.069298,1.0,6.0,1.091501,-1.165919,105613.399742,105613.399742,1.438643,1.438643


In [27]:
df.to_excel('crypto.xlsx',sheet_name='CryptoData',index=False)